# Pointwise Ordinal Regression

In this notebook, we examine the pointwise ordinal regression approach, including tuning. First, we load the required dependencies and the data.

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd
from category_encoders.binary import BinaryEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from category_encoders.target_encoder import TargetEncoder
from datetime import timedelta
from sklearn.compose import ColumnTransformer
from time import time

from src import configuration as config
from src.pipeline.pipeline_factory import PipelineFactory, ModelType, EvaluationType
from src.pipeline.pipeline_transformers import *


# load the data
train_df = config.load_traindata_for_pointwise()
pipelineFactory = PipelineFactory()

## Pointwise Ordinal Regression

In [4]:
start = time()

pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_ORDINAL_REGRESSION_NO_SEARCH,
    evaluation=EvaluationType.CROSS_VALIDATION,
    verbose_level=1,
    n_folds=5,
    workers=1,
    target="rank"
)

pipeline.run()

runtime = int(time() - start)
print('\nruntime: ' + str(timedelta(seconds=runtime)) + ' [' + str(runtime) + 's]')

Creating pipeline ...
Starting pipeline using method: EvaluationType.CROSS_VALIDATION


100%|█████████████████████████████████████████████| 5/5 [01:12<00:00, 14.51s/it]

Finished running the pipeline
Evaluation metrics:
    validation_average_spearman_fold_0: 0.7479
    validation_average_spearman_fold_1: 0.7569
    validation_average_spearman_fold_2: 0.7275
    validation_average_spearman_fold_3: 0.793
    validation_average_spearman_fold_4: 0.7676
    average of all folds: 0.7586 [std=0.0217]

runtime: 0:01:25 [85s]


In [5]:
start = time()

param_grid = {
    "encoder_transformer__batch_size": [50, 500],
}

pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_ORDINAL_REGRESSION_NO_SEARCH,
    verbose_level=1,
    evaluation=EvaluationType.GRID_SEARCH,
    param_grid=param_grid,
    n_folds=2,
    workers=1,
    target="rank"
)

pipeline.run()

runtime = int(time() - start)
print('\nruntime: ' + str(timedelta(seconds=runtime)) + ' [' + str(runtime) + 's]')

Creating pipeline ...
Starting pipeline using method: EvaluationType.GRID_SEARCH
Performing grid search


100%|█████████████████████████████████████████████| 2/2 [00:55<00:00, 27.83s/it]

Finished running the pipeline
Evaluation metrics:
    best_score: 0.7587
    best_params: {'encoder_transformer__batch_size': 50}

runtime: 0:00:55 [55s]


### Tuning with Bayes Search

In [ ]:
start = time()

# number of optimization rounds = n_iter / n_points (e.g. 50 rounds in our case)
n_iter = 200 # how many unique parameters to examine - our default: 200
n_points = 4 # how many unique parameter combinations per optimization round - our default: 4
cv = 4 # how many fits for each unique parameter combination - our default: 4
n_jobs = -1 # how many fits in parallel (only parallelizable per round) - our default: -1

pipeline = pipelineFactory.create_pipeline(
    train_df,
    ModelType.POINTWISE_ORDINAL_REGRESSION_BAYES_SEARCH,
    verbose_level=1,
    target="rank",
    bayes_n_iter=n_iter,
    bayes_n_points=n_points,
    bayes_cv=cv,
    bayes_n_jobs=n_jobs
)

pipeline.run()

runtime = int(time() - start)
print('\nruntime: ' + str(timedelta(seconds=runtime)) + ' [' + str(runtime) + 's]')